In [13]:
import numpy as np
import operator

from qiskit import Aer, execute, QuantumCircuit
from qiskit.visualization import plot_histogram

from ipynb.fs.defs.qft import QFT

In [14]:
class QPE:
    def __init__(self, n_measurement_qubits, theta):
        self.theta = theta
        self._n_measurement_qubits = n_measurement_qubits
        self._n_all_qubits = n_measurement_qubits + 1
        
        self._measurement_qubits = range(self._n_measurement_qubits)
        self._target_qubit = self._n_all_qubits - 1
        self._all_qubits = range(self._n_all_qubits)
        
        self.circuit = QuantumCircuit(self._n_all_qubits, self._n_measurement_qubits)
        self.circuit.circuit_name = 'QPE'
        
        self._create_qpe_circuit()
        
    def _create_qpe_circuit(self):
        self._initialization()
        self._add_unitary_exponentiations()
        self._add_inverse_qft()
        self.circuit.measure(self._measurement_qubits, self._measurement_qubits)
    
    def _initialization(self):
        self.circuit.h(self._measurement_qubits)
        self.circuit.x(self._target_qubit)
        
    def _add_unitary_exponentiations(self):
        for control_qubit in self._measurement_qubits:
            self._unitary_exponetiations(control_qubit, self._target_qubit)
            
    def _add_inverse_qft(self):
        qft_circuit = QFT(self._n_measurement_qubits).circuit
        self.circuit.append(qft_circuit.inverse(), self._measurement_qubits)

    def _unitary_exponetiations(self, control, target):
        exponent = 2**(self._n_measurement_qubits - control - 1)
        self.circuit.cu1(exponent * 2 * np.pi * self.theta, control, target)
        
    def run_program(self):
        simulator = Aer.get_backend('qasm_simulator')
        return execute(self.circuit, backend=simulator, shots=1000).result()
    
    def get_theta_from_counts(self, counts):
        highest_probability_outcome = max(counts.items(), key=operator.itemgetter(1))[0][::-1]
        theta_measured = int(highest_probability_outcome, 2)/2**self._n_measurement_qubits
        return theta_measured

In [21]:
theta = 0.123
n_measurement_qubits = 10

qpe = QPE(n_measurement_qubits, theta)

display(qpe.circuit.draw())

┌───┐                                                               »
 q_0: ┤ H ├─■─────────────────────────────────────────────────────────────»
      ├───┤ │                                                             »
 q_1: ┤ H ├─┼────────■────────────────────────────────────────────────────»
      ├───┤ │        │                                                    »
 q_2: ┤ H ├─┼────────┼────────■───────────────────────────────────────────»
      ├───┤ │        │        │                                           »
 q_3: ┤ H ├─┼────────┼────────┼────────■──────────────────────────────────»
      ├───┤ │        │        │        │                                  »
 q_4: ┤ H ├─┼────────┼────────┼────────┼────────■─────────────────────────»
      ├───┤ │        │        │        │        │                         »
 q_5: ┤ H ├─┼────────┼────────┼────────┼────────┼────────■────────────────»
      ├───┤ │        │        │        │        │        │                »
 q_6: ┤ H ├─┼────────┼────────┼────────┼────────┼────────┼────────■───────»
      ├───┤ │        │        │        │        │        │        │       »
 q_7: ┤ H ├─┼────────┼────────┼────────┼────────┼────────┼────────┼───────»
      ├───┤ │        │        │        │        │        │        │       »
 q_8: ┤ H ├─┼────────┼────────┼────────┼────────┼────────┼────────┼───────»
      ├───┤ │        │        │        │        │        │        │       »
 q_9: ┤ H ├─┼────────┼────────┼────────┼────────┼────────┼────────┼───────»
      ├───┤ │395.69  │197.84  │98.922  │49.461  │24.731  │12.365  │6.1827 »
q_10: ┤ X ├─■────────■────────■────────■────────■────────■────────■───────»
      └───┘                                                               »
c: 10/════════════════════════════════════════════════════════════════════»
                                                                          »
«                                  ┌────────────────┐┌─┐                     »
« q_0: ────────────────────────────┤0               ├┤M├─────────────────────»
«                                  │                │└╥┘┌─┐                  »
« q_1: ────────────────────────────┤1               ├─╫─┤M├──────────────────»
«                                  │                │ ║ └╥┘┌─┐               »
« q_2: ────────────────────────────┤2               ├─╫──╫─┤M├───────────────»
«                                  │                │ ║  ║ └╥┘┌─┐            »
« q_3: ────────────────────────────┤3               ├─╫──╫──╫─┤M├────────────»
«                                  │                │ ║  ║  ║ └╥┘┌─┐         »
« q_4: ────────────────────────────┤4               ├─╫──╫──╫──╫─┤M├─────────»
«                                  │  circuit104_dg │ ║  ║  ║  ║ └╥┘┌─┐      »
« q_5: ────────────────────────────┤5               ├─╫──╫──╫──╫──╫─┤M├──────»
«                                  │                │ ║  ║  ║  ║  ║ └╥┘┌─┐   »
« q_6: ────────────────────────────┤6               ├─╫──╫──╫──╫──╫──╫─┤M├───»
«                                  │                │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
« q_7: ─■──────────────────────────┤7               ├─╫──╫──╫──╫──╫──╫──╫─┤M├»
«       │                          │                │ ║  ║  ║  ║  ║  ║  ║ └╥┘»
« q_8: ─┼────────■─────────────────┤8               ├─╫──╫──╫──╫──╫──╫──╫──╫─»
«       │        │                 │                │ ║  ║  ║  ║  ║  ║  ║  ║ »
« q_9: ─┼────────┼────────■────────┤9               ├─╫──╫──╫──╫──╫──╫──╫──╫─»
«       │3.0913  │1.5457  │0.77283 └────────────────┘ ║  ║  ║  ║  ║  ║  ║  ║ »
«q_10: ─■────────■────────■───────────────────────────╫──╫──╫──╫──╫──╫──╫──╫─»
«                                                     ║  ║  ║  ║  ║  ║  ║  ║ »
«c: 10/═══════════════════════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩═»
«                                                     0  1  2  3  4  5  6  7 »
«            
« q_0: ──────
«            
« q_1: ──────
«            
« q_2: ──────
«            
« q_3: ──────
«            
« q_4

In [22]:
result = qpe.run_program()
counts = result.get_counts(qpe.circuit)
plot_histogram(counts)

theta_measured = qpe.get_theta_from_counts(counts)

print('The measured value of theta is {theta_measured}'.format(theta_measured=theta_measured))

The measured value of theta is 0.123046875
